In [ ]:
import pandas as pd
import numpy as np
import random
from PIL import ImageStat
import h5py

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils

# run jupyter with THEANO_FLAGS='floatX=float32,device=gpu'
# or include 
#   [global]
#   floatX = float32
#   device = gpu0
# to ~/.theanorc

# Also, add 
#   [cuda]
#   root = /usr/local/cuda
# to ~/.theanorc


In [ ]:
batch_size = 1
nb_epoch = 50
samples_per_epoch = 10000

## Load, filter and label location data

In [ ]:
locations = pd.read_hdf("data/backup/complete_paths.h5", "data")

In [ ]:
locations = locations[((locations['ice'] > 0) & (locations['ice'] <= 100)) | (locations["ice"] == 255)]

In [ ]:
def building_classes(ice_conc):
    if ice_conc > 0 and ice_conc <= 100: # ten classes of ice concentration
        return 1#int(np.ceil(ice_conc/10))
    #elif ice_conc == 255: # ocean
    #    return 0
    else:  # coast
        return 0# 11

In [ ]:
locations["labels"] = locations["ice"].map(building_classes)

In [ ]:
# perceived brightness
def brightness(img):
    stat = ImageStat.Stat(img)
    r,g,b = stat.mean
    return np.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

def brightness_gray(img):
    stat = ImageStat.Stat(img)
    return stat.mean[0]

def train_generator(images, labels, batch_size=32):
    
    #train_datagen = ImageDataGenerator(
    #    rescale=1./255,
    #    shear_range=0.2,
    #    zoom_range=0.2,
    #    horizontal_flip=True)
    labels = np_utils.to_categorical(labels, nb_classes=np.max(labels) + 1)
    
    i = -1
    while True:
        batch = np.ndarray((batch_size, 3, 192, 256))
        batch_label = np.ndarray((batch_size, 2))
        
        step = 0
        while step < batch_size:
            i = (i+1) % len(images)
            #i = random.randrange(len(images))
            try:
                image = load_img(images[i], target_size=(192, 256))
            except FileNotFoundError:
                continue
            
            # only use images with sufficient brightness, margin choosen by fair dice rolls
            if brightness(image) > 70.0:
                batch[step] = img_to_array(image)/255#.reshape(1,3,192,256)/255
                batch_label[step] = labels[i]
                step += 1
            else:
                continue
                
        #print(i, step)
        
        yield batch, batch_label

In [ ]:
n = 0
for x,y in train_generator(locations["image"], locations["labels"], model, batch_size=1):
    
    print(x.shape,y.shape)
    n += 1
    
    if n == 1:
        break

In [ ]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3,192,256)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(128, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(256, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
generator = train_generator(locations["image"], locations["labels"], batch_size=batch_size)

In [ ]:
model.summary()
model.fit_generator(generator, samples_per_epoch=10000, nb_epoch=50)

In [ ]:
#x = load_img("data/images/remote/20111120-2201.jpeg", target_size=(192, 256))
x = load_img("data/images/remote/20111120-2201.jpeg")
#ground = 

#y = model.predict(img_to_array(x).reshape(1,3,192,256)/255)[0]
x